In [11]:
import pandas as pd
import numpy as np
import matplotlib as plt
from IPython.display import display
import xgboost as xgb

print('loading aisles...')
aisles = pd.read_csv('aisles.csv', dtype={
        'aisle_id': np.uint16,
        'aisle': 'category'})

print('loading department...')
department = pd.read_csv('departments.csv', dtype={
            'department_id': np.uint8,
            'department': 'category'})

print('loading products...')
products = pd.read_csv('products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.uint32,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

print('loading prior orders...')
prior = pd.read_csv('order_products__prior.csv', dtype={
        'order_id': np.uint32,
        'product_id': np.uint16,
        'add_to_cart_order': np.uint16,
        'reordered': np.uint16})

print('loading train orders...')
train = pd.read_csv('order_products__train.csv', dtype={
        'order_id': np.uint32,
        'product_id': np.uint16,
        'add_to_cart_order': np.uint16,
        'reordered': np.uint8})

print('loading orders...')
order = pd.read_csv('orders.csv' , dtype={
        'order_id': np.uint32,
        'user_id': np.uint32,
        'eval_set': 'category',
        'order_number': np.uint16,
        'order_dow': np.uint16,
        'order_hour_of_day': np.uint16,
        'days_since_prior_order': np.float32})

loading aisles...
loading department...
loading products...
loading prior orders...
loading train orders...
loading orders...


In [20]:
train_orders = order[order.eval_set == 'train'].copy()
test_orders = order[order.eval_set == 'test'].copy()
prior_orders = order[order.eval_set == 'prior'].copy()
train_prior_orders = pd.concat([train_orders, prior_orders], ignore_index=True)
train_prior_orders_detail = pd.concat([prior, train], ignore_index=True)
display(train_prior_orders_detail)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [16]:
# user global features
global_features = pd.DataFrame()
global_features['order_id'] = train_prior_orders['order_id']
global_features['user_id'] = train_prior_orders['user_id']
global_features['user_nb_orders'] = train_prior_orders.groupby(global_features.user_id).size().astype(np.uint16)

collect_user_info(prior_orders, labels_out=True)
collect_user_info(train_orders)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'